# Integrantes del equipo Concentrados
- Daniel Queijeiro Albo - A01710441
- Diego Alfaro Pinto - A01709971
- Diego Isaac Fuentes Juvera - A01705506
- Jesus Ramirez Delgado - A01274723
- Mauricio Anguiano Juarez - A01703337
- Luis Adrián Uribe Cruz - A01783129

## TC 3007B
### GPT 2

<br>

#### Activity 2,3: Code GPT2
<br>

- Objective:
    - To understand the Transformer architecture.
    - To code GPT 2.
    - To gain understanding of the LLMs' autoregresive nature..
    
<br>

- Instructions

    This activity requires submission in teams. While teamwork is encouraged, each member is expected to contribute individually to the assignment. The final submission should feature the best arguments and solutions from each team member. Only one person per team needs to submit the completed work, but it is imperative that the names of all team members are listed in a Markdown cell at the very beginning of the notebook (either the first or second cell). Failure to include all team member names will result in the grade being awarded solely to the individual who submitted the assignment, with zero points given to other team members (no exceptions will be made to this rule).

    Follow the provided code. The code already implements a transformer from scratch as explained in [this video](https://youtu.be/51jq4wnHYaY)

    Since the provided code already implements a simple translator, your job for this assignment is to understand it fully, and document it using pictures, figures, and markdown cells.  
  
- Evaluation Criteria

    - Code Readability and Comments (40%).
    - Traning a LM,  complete 'Train function' (30%).
    - Generating at least 10 sentences, comple 'Sample function' (30%).

- Submission

Submit this Jupyter Notebook in canvas with your complete solution, ensuring your code is well-commented and includes Markdown cells that explain your design choices, results, and any challenges you encountered.




# Contribuciones individuales
Daniel Queijeiro - Buscar datasets e importarlos para el entrenamiento
Diego Fuentes - Proponer el tokenizer Tiktoken para experimentar y mejorar el modelo
Diego Alfaro - Implementar la función train
Mauricio Anguiano - Responsable de implementar GPU para el entrenamiento
Jesus Ramirez - Implementar la función sample
Luis Adrián - Comentar código

In [ ]:
!pip install transformers datasets

In [ ]:
# Importar las bibliotecas necesarias para construir y entrenar el modelo GPT-2
import torch                                    # Biblioteca principal de PyTorch para operaciones con tensores
import torch.nn as nn                           # Módulos de redes neuronales (capas, funciones de pérdida, etc.)
import torch.nn.functional as F                 # API funcional para funciones de activación, softmax, etc.
from torch.utils.data import Dataset, DataLoader  # Utilidades para manejar conjuntos de datos y lotes
from datasets import load_dataset              # Biblioteca de Hugging Face para cargar datasets predefinidos
import torch.optim as optim                    # Algoritmos de optimización (Adam, SGD, etc.)

In [ ]:
class Config:
    '''
    Clase de configuración que contiene todos los hiperparámetros del modelo GPT-2.
    Estos parámetros definen la arquitectura y el comportamiento del transformer.

    Atributos:
        vocab_size: Tamaño del vocabulario (por defecto 50257 para el tokenizador GPT-2)
        max_seq_length: Longitud máxima de secuencia que el modelo puede procesar
        embed_size: Dimensión de los embeddings de tokens y estados ocultos
        num_layers: Número de bloques transformer apilados
        num_heads: Número de cabezas de atención en la atención multi-cabeza
        dropout: Probabilidad de dropout para regularización
    '''
    def __init__(self, vocab_size = 50257, max_seq_length = 128, embed_size = 768, num_layers = 12,
                 num_heads = 12, dropout = 0.1):
        self.vocab_size = vocab_size          # GPT-2 usa tokenización BPE con ~50k tokens
        self.max_seq_length = max_seq_length  # Tamaño de la ventana de contexto
        self.embed_size = embed_size          # Dimensión oculta (d_model en el paper)
        self.num_layers = num_layers          # Profundidad del transformer
        self.num_heads = num_heads            # Operaciones de atención en paralelo
        self.dropout = dropout                # Regularización para prevenir sobreajuste

In [ ]:
class SelfAttention(nn.Module):
    '''
    Mecanismo de Auto-Atención Multi-Cabeza con enmascaramiento causal para generación autoregresiva.

    Este es el componente central del transformer que permite a cada token atender
    a los tokens anteriores en la secuencia. La máscara causal asegura que las predicciones
    para la posición i solo dependan de las posiciones < i (propiedad autoregresiva).

    La fórmula de atención es: Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V
    '''

    def __init__(self, config):
        super().__init__()
        # Asegurar que embed_size sea divisible por num_heads para división equitativa
        assert config.embed_size % config.num_heads == 0, 'tamaños no compatibles'
        self.num_heads = config.num_heads
        self.head_dim = config.embed_size // config.num_heads  # Dimensión por cabeza de atención

        # Matrices de proyección lineal para Query, Key y Value
        # Estas aprenden a proyectar los embeddings de entrada a los espacios Q, K, V
        self.W_q = nn.Linear(config.embed_size, config.embed_size)  # Proyección de Query
        self.W_k = nn.Linear(config.embed_size, config.embed_size)  # Proyección de Key
        self.W_v = nn.Linear(config.embed_size, config.embed_size)  # Proyección de Value
        self.output = nn.Linear(config.embed_size, config.embed_size)  # Proyección de salida
        self.dropout = nn.Dropout(config.dropout)

        # Crear máscara causal (matriz triangular inferior) para prevenir atención a tokens futuros
        # Esta máscara se registra como buffer (no como parámetro) para que se guarde con el modelo
        # pero no se actualice durante el entrenamiento
        self.register_buffer(
            'mask',
            torch.tril(torch.ones(config.max_seq_length, config.max_seq_length)
                      ).view(-1, 1, config.max_seq_length, config.max_seq_length)
        )

    def forward(self, x):
        batch, seq_length, embed_dim = x.size()  # B = tamaño de lote, T = longitud de secuencia, D = dimensión de embedding

        # Proyectar entrada a Q, K, V y reorganizar para atención multi-cabeza
        # Transformación de forma: (B, T, D) -> (B, T, num_heads, head_dim) -> (B, num_heads, T, head_dim)
        Q = self.W_q(x).view(batch, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(x).view(batch, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(x).view(batch, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

        # Calcular puntuaciones de atención con producto punto escalado
        # QK^T / sqrt(d_k) - el escalado previene que los gradientes se vuelvan muy pequeños
        attn = (Q @ K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # Forma: (B, num_heads, T, T)

        # Aplicar máscara causal: establecer posiciones futuras a -inf para que softmax les dé probabilidad 0
        attn = attn.masked_fill(self.mask[:, :, :seq_length, :seq_length] == 0, float('-inf'))

        # Convertir puntuaciones de atención a probabilidades
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)

        # Calcular suma ponderada de valores basada en probabilidades de atención
        scores = attn @ V  # Forma: (B, num_heads, T, head_dim)

        # Concatenar cabezas y proyectar de vuelta a embed_size
        # Forma: (B, num_heads, T, head_dim) -> (B, T, num_heads, head_dim) -> (B, T, embed_size)
        scores = scores.transpose(1, 2).contiguous().view(batch, seq_length, embed_dim)

        return self.dropout(scores)

In [ ]:
class FFN(nn.Module):
    '''
    Red Feed-Forward (FFN) - también conocida como bloque MLP en el transformer.

    Esta es una red totalmente conectada aplicada posición por posición después de la atención.
    Consiste en dos transformaciones lineales con una activación GELU en medio.
    La capa oculta expande a 4x el tamaño del embedding, luego proyecta de vuelta.

    FFN(x) = GELU(xW1 + b1)W2 + b2

    La expansión permite al modelo aprender transformaciones no lineales más complejas.
    '''
    def __init__(self, config):
        super().__init__()
        # Primera capa lineal expande la dimensión por 4x (estándar en GPT-2)
        self.fc1 = nn.Linear(config.embed_size, 4 * config.embed_size)
        # Activación GELU (Unidad Lineal de Error Gaussiano) - más suave que ReLU
        self.gelu = nn.GELU()
        # Segunda capa lineal proyecta de vuelta a la dimensión original
        self.fc2 = nn.Linear(4 * config.embed_size, config.embed_size)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        # Aplicar expansión -> activación -> proyección -> dropout
        x = self.fc2(self.gelu(self.fc1(x)))
        return self.dropout(x)

In [ ]:
class Transformer(nn.Module):
    '''
    Bloque Transformer individual que combina Auto-Atención y Red Feed-Forward.

    Esto implementa la variante Pre-LayerNorm (usada en GPT-2) donde LayerNorm
    se aplica antes de cada sub-capa en lugar de después. Esto proporciona un
    entrenamiento más estable comparado con el diseño Post-LayerNorm original.

    Estructura:
        x -> LayerNorm -> Auto-Atención -> + (residual) -> LayerNorm -> FFN -> + (residual)

    Las conexiones residuales ayudan con el flujo de gradientes en redes profundas.
    '''
    def __init__(self, config):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.embed_size)  # Normalizar antes de atención
        self.attention = SelfAttention(config)         # Auto-atención multi-cabeza
        self.norm2 = nn.LayerNorm(config.embed_size)  # Normalizar antes de FFN
        self.mlp = FFN(config)                         # Red feed-forward

    def forward(self, x):
        # Sub-bloque de atención con conexión residual
        # x + Attention(LayerNorm(x))
        x = x + self.attention(self.norm1(x))
        # Sub-bloque FFN con conexión residual
        # x + FFN(LayerNorm(x))
        x = x + self.mlp(self.norm2(x))
        return x

In [ ]:
class GPT2(nn.Module):
    '''
    Arquitectura completa del Modelo de Lenguaje GPT-2.

    Este modelo combina:
    1. Embeddings de tokens - convierte IDs de tokens a vectores densos
    2. Embeddings posicionales - codifica información de posición (aprendidos, no sinusoidales)
    3. Pila de bloques Transformer - procesa la secuencia
    4. Cabeza de modelado de lenguaje - proyecta de vuelta al vocabulario para predicción del siguiente token

    Los logits de salida representan la distribución de probabilidad sobre el vocabulario
    para el siguiente token en cada posición.
    '''
    def __init__(self, config):
        super().__init__()
        self.config = config

        # Embedding de tokens: mapea índices del vocabulario a vectores densos
        self.token_embed = nn.Embedding(config.vocab_size, config.embed_size)

        # Embedding posicional: codificaciones de posición aprendidas (a diferencia del sinusoidal del Transformer original)
        self.pos_embed = nn.Embedding(config.max_seq_length, config.embed_size)

        self.dropout = nn.Dropout(config.dropout)

        # Pila de bloques transformer - aquí es donde ocurre la "magia"
        self.transformers = nn.Sequential(*[Transformer(config) for _ in range(config.num_layers)])

        # Normalización de capa final antes de la proyección de salida
        self.norm1 = nn.LayerNorm(config.embed_size)

    def forward(self, input_tokens):
        batch, seq_length = input_tokens.size()

        # Crear índices de posición [0, 1, 2, ..., seq_length-1]
        pos = torch.arange(0, seq_length, dtype=torch.long, device=input_tokens.device).unsqueeze(0)

        # Combinar embeddings de tokens y embeddings posicionales
        # Esto da a cada token tanto su significado como su información de posición
        x = self.token_embed(input_tokens) + self.pos_embed(pos)
        x = self.dropout(x)

        # Pasar a través de todas las capas transformer
        x = self.transformers(x)

        # Aplicar normalización de capa final
        x = self.norm1(x)

        # Proyectar al tamaño del vocabulario usando weight tying (reutilizar pesos del embedding de tokens)
        # Esta es una técnica común que reduce parámetros y frecuentemente mejora el rendimiento
        # Forma de salida: (batch, seq_length, vocab_size)
        return x @ self.token_embed.weight.t()

In [ ]:
# Importar el tokenizador pre-entrenado de GPT-2 de Hugging Face
from transformers import GPT2Tokenizer

# Configurar dispositivo a GPU si está disponible, de lo contrario usar CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar el tokenizador GPT-2 - maneja la conversión de texto a IDs de tokens y viceversa
# El tokenizador usa Codificación de Pares de Bytes (BPE) con un vocabulario de ~50,257 tokens
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')

# Establecer el token de relleno igual al token de fin de secuencia
# GPT-2 no tiene un token de relleno dedicado, así que reutilizamos eos_token
tokeniser.pad_token = tokeniser.eos_token

# Definir longitud de secuencia para entrenamiento (tamaño de ventana de contexto)
SEQ_LENGTH = 128

# Inicializar la configuración del modelo y crear el modelo GPT-2
config = Config(max_seq_length=SEQ_LENGTH)
model = GPT2(config).to(device)  # Mover modelo a GPU/CPU

# Inicializar optimizador AdamW con tasa de aprendizaje 3e-4
# AdamW es Adam con decaimiento de peso apropiado (desacoplado de las actualizaciones de gradiente)
optimiser = optim.AdamW(model.parameters(), lr=3e-4)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

## Tokenizador GPT-2

### ¿Por qué usamos el tokenizador de GPT-2?

El **tokenizador GPT-2** utiliza el algoritmo de **Byte-Pair Encoding (BPE)**, que es fundamental para el procesamiento eficiente del lenguaje natural. Las razones principales para usar este tokenizador son:

1. **Codificación de Pares de Bytes (BPE)**:
   - BPE es un algoritmo de compresión que aprende a dividir palabras en subpalabras frecuentes
   - Maneja palabras desconocidas dividiéndolas en subunidades conocidas
   - Balance óptimo entre vocabulario de caracteres (muy largo) y vocabulario de palabras (incompleto)

2. **Vocabulario de ~50,257 tokens**:
   - Tamaño suficiente para capturar la mayoría de patrones lingüísticos
   - Incluye tokens para caracteres especiales, puntuación y espacios

3. **Compatibilidad**:
   - Usar el mismo tokenizador que el modelo original GPT-2 asegura consistencia
   - Permite aprovechar el vocabulario pre-entrenado de OpenAI

4. **Multilingüe**:
   - Aunque fue entrenado principalmente en inglés, BPE puede manejar texto en español
   - Las subpalabras permiten representar palabras en otros idiomas

### Diagrama del proceso de tokenización:

```
Texto: "Hola mundo"
         ↓
    [Tokenización BPE]
         ↓
IDs: [39, 5765, 416, 78, 1597]
         ↓
    [Embedding]
         ↓
Vectores densos de dimensión 768
```

In [ ]:
# Cargar y preparar el dataset de texto en español para entrenamiento

# URL a un corpus de texto en español del dataset MC4 (C4 multilingüe)
# Esta es una versión limpia de datos web de Common Crawl en español
data_files = "https://huggingface.co/datasets/bertin-project/mc4-es-sampled/resolve/main/mc4-es-train-50M-gaussian-shard-0001-of-1024.json.gz"

# Cargar el dataset usando la biblioteca datasets de Hugging Face
# El constructor 'json' parsea el archivo JSON comprimido
# split="train[:100%]" carga todo el split de entrenamiento
dataset = load_dataset("json", data_files=data_files, split="train[:100%]")

def tokenize_function(examples):
    '''
    Tokeniza ejemplos de texto en IDs de tokens.

    Args:
        examples: Diccionario conteniendo el campo 'text' con cadenas de texto crudo

    Returns:
        Diccionario con 'input_ids' conteniendo secuencias tokenizadas
    '''
    # Tokenizar, truncar a longitud máxima y rellenar secuencias más cortas
    return tokeniser(examples["text"], truncation=True, max_length=SEQ_LENGTH, padding="max_length")

# Aplicar tokenización a todo el dataset (en lotes para eficiencia)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Establecer el formato del dataset a tensores de PyTorch y mantener solo input_ids
tokenized_dataset.set_format(type='torch', columns=['input_ids'])

# Crear un DataLoader para dividir en lotes y mezclar durante el entrenamiento
# batch_size=8: Procesar 8 secuencias a la vez
# shuffle=True: Aleatorizar el orden en cada época para mejor generalización
loader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)

Map:   0%|          | 0/48829 [00:00<?, ? examples/s]

## Dataset MC4 en Español

### ¿Por qué usamos el dataset MC4-ES?

El **MC4 (Multilingual Colossal Clean Crawled Corpus)** es una versión multilingüe del dataset C4, específicamente diseñado para entrenar modelos de lenguaje. Elegimos la versión en español por las siguientes razones:

1. **Datos de alta calidad**:
   - Proviene de Common Crawl, pero con limpieza extensiva
   - Filtrado para remover contenido duplicado, texto de baja calidad y spam
   - Mantenido por el proyecto BERTIN, especializado en NLP en español

2. **Volumen de datos**:
   - Contiene millones de textos en español
   - Suficiente para entrenar un modelo de lenguaje desde cero
   - Variedad de dominios: noticias, blogs, wikipedia, etc.

3. **Formato accesible**:
   - Disponible en Hugging Face Datasets
   - Formato JSON comprimido para descarga eficiente
   - Fácil integración con el ecosistema de PyTorch

4. **Relevancia para el proyecto**:
   - Entrenar con texto en español demuestra la capacidad del modelo
   - Permite generar texto coherente en nuestro idioma
   - Útil para aplicaciones de NLP en español

### Estructura del dataset:

| Campo | Descripción |
|-------|-------------|
| `text` | Contenido textual del documento |
| `url` | URL de origen del texto |
| `timestamp` | Fecha de extracción |

### Pipeline de procesamiento:

```
URL del dataset (JSON.gz)
         ↓
    [load_dataset]
         ↓
    Dataset crudo
         ↓
    [tokenize_function]
         ↓
    Secuencias de IDs
         ↓
    [DataLoader]
         ↓
    Lotes para entrenamiento
```

In [ ]:
def train(model, loader, optimiser, epochs=10):
    '''
    Bucle de entrenamiento para el modelo de lenguaje GPT-2.

    Usa el objetivo estándar de modelado de lenguaje: predecir el siguiente token
    dados todos los tokens anteriores. Esto es entrenamiento autoregresivo.

    Args:
        model: El modelo GPT-2 a entrenar
        loader: DataLoader que proporciona lotes de texto tokenizado
        optimiser: Optimizador para actualizar los pesos del modelo
        epochs: Número de pasadas completas a través del dataset
    '''
    model.train()  # Establecer modelo en modo entrenamiento (habilita dropout, etc.)

    for epoch in range(epochs):
        total_loss = 0.0

        for batch in loader:
            # Mover tokens de entrada al dispositivo apropiado (GPU/CPU)
            input_ids = batch['input_ids'].to(device)

            # Desplazamiento para entrenamiento autoregresivo:
            # Entrada: tokens [0, 1, 2, ..., n-1] (todos menos el último)
            # Objetivo: tokens [1, 2, 3, ..., n] (todos menos el primero)
            # Esto enseña al modelo a predecir cada siguiente token
            inputs = input_ids[:, :-1]   # Todo excepto el último token
            targets = input_ids[:, 1:]   # Todo excepto el primer token

            optimiser.zero_grad()  # Limpiar gradientes del paso anterior

            # Paso forward: obtener logits para cada posición
            logits = model(inputs)  # Forma: (batch, seq_length-1, vocab_size)

            # Calcular pérdida de entropía cruzada
            # Aplanar logits y objetivos para la función de pérdida
            # logits: (batch * seq_length, vocab_size)
            # targets: (batch * seq_length,)
            loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))

            # Paso backward: calcular gradientes
            loss.backward()

            # Actualizar pesos usando los gradientes calculados
            optimiser.step()

            total_loss += loss.item()

        # Imprimir pérdida promedio para esta época
        print(f"Época {epoch+1}/{epochs}, Pérdida: {total_loss/len(loader):.4f}")

In [ ]:
# Iniciar entrenamiento del modelo por 10 épocas
# Esto tomará algo de tiempo dependiendo del tamaño del dataset y el hardware
train(model, loader, optimiser, epochs=10)

Epoch 1/10, Loss: 8.1781
Epoch 2/10, Loss: 4.6126
Epoch 3/10, Loss: 4.0241
Epoch 4/10, Loss: 3.6984
Epoch 5/10, Loss: 3.5213
Epoch 6/10, Loss: 3.3912
Epoch 7/10, Loss: 3.2870
Epoch 8/10, Loss: 3.1976
Epoch 9/10, Loss: 3.1184
Epoch 10/10, Loss: 3.0457


In [ ]:
def sample(model, device, tokenizer, prompt, length=50, temperature=1.0):
    '''
    Genera texto de forma autoregresiva desde el modelo entrenado.

    Comenzando desde un prompt, el modelo predice un token a la vez,
    agregando cada token predicho a la secuencia y repitiendo.

    Args:
        model: Modelo GPT-2 entrenado
        device: Dispositivo para ejecutar inferencia (cuda/cpu)
        tokenizer: Tokenizador para codificar/decodificar texto
        prompt: Texto inicial desde donde comenzar la generación
        length: Número de nuevos tokens a generar
        temperature: Controla la aleatoriedad (1.0 = normal, <1 = más enfocado, >1 = más aleatorio)

    Returns:
        Texto generado como cadena
    '''
    model.eval()  # Establecer modelo en modo evaluación (deshabilita dropout)

    # Codificar el prompt en IDs de tokens
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generar tokens uno a la vez (generación autoregresiva)
    for _ in range(length):
        # Solo usar los últimos SEQ_LENGTH tokens si la secuencia se vuelve muy larga
        tokens_cond = tokens[:, -SEQ_LENGTH:]

        # Obtener predicciones del modelo sin calcular gradientes (más rápido)
        with torch.no_grad():
            logits = model(tokens_cond)

        # Obtener logits para la última posición y aplicar escalado de temperatura
        # Mayor temperatura = más aleatorio, menor = más determinístico
        next_token_logits = logits[:, -1, :] / temperature

        # Muestrear de la distribución de probabilidad (no argmax para diversidad)
        # Esto da variedad en el texto generado en lugar de siempre elegir el token más probable
        next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)

        # Agregar el token predicho a la secuencia
        tokens = torch.cat([tokens, next_token], dim=1)

    print(tokens)  # Imprimir IDs de tokens crudos para depuración

    # Decodificar todos los tokens de vuelta a texto
    return tokenizer.decode(tokens[0])

# Ejemplo de uso: Generar texto comenzando con un prompt en español
print(sample(model, device, tokeniser, prompt="Un estudiante de doctorado", length=50))

tensor([[ 3118,  1556,   463,  3014,    68,   390,  6253,  4533, 17605,    68,
          2382,   291,  5733,  8591,  4902,   304,    73,   721,   315, 12151,
           390,  8591, 26986, 32482,  1619, 26482,   930,  2295,   349,    13,
           785,   198,  3118,  1556,   463,  3014,    68,   390,  6253,  4533,
         17605,    68,  2382,   291,  5733,  8591,  4902,   304,    73,   721,
           315, 12151,   390,  8591, 26986, 32482,  1619, 26482]],
       device='cuda:0')
Un estudiante de doctorado norteamericano la crisis ejecutiva de la Universidad del Salvador | Emol.com
Un estudiante de doctorado norteamericano la crisis ejecutiva de la Universidad del Salvador


In [ ]:
# Generar 10 oraciones usando diferentes prompts en español
# Esto demuestra la capacidad del modelo para continuar texto desde varios puntos de partida

prompts = [
    "Un ingeniero",
    "La inteligencia artificial",
    "El futuro de la tecnología",
    "En el año 2050",
    "Los científicos descubrieron",
    "La universidad ofrece",
    "El presidente anunció",
    "Durante la conferencia",
    "Los investigadores encontraron",
    "El nuevo sistema"
]

# Generar y mostrar texto para cada prompt
for i, prompt in enumerate(prompts, 1):
    print(f"\n--- Oración {i} ---")
    print(f"Prompt: {prompt}")
    # Generar 50 tokens de continuación para cada prompt
    print(f"Generado: {sample(model, device, tokeniser, prompt=prompt, length=50)}")


--- Sentence 1 ---
Prompt: Un ingeniero
tensor([[ 3118, 27016,   959,    78,   285, 40138,   551,   325, 12654,   283,
           257,   424,  2614, 32482,  2968,   257,  9223,   544,   532, 20442,
            78,  4496,  2947,   198,  1415,  2362, 26597,  4679,  1853, 16964,
          2297, 44456, 18840,   978, 40407,    78,    13,  2398,   198,  9527,
         27016,   959,    78,  2604,    81, 10205,   435, 46436, 23593,   257,
         21733,   395,  8847,  1145]], device='cuda:0')
Generated: Un ingeniero más enseñar a su personalidad popular a Rusia - Mendoza Post
14 Septiembre 2015 por Redacción Al Socialismo.org
El ingeniero logró al positivo a Nuestras fam

--- Sentence 2 ---
Prompt: La inteligencia artificial
tensor([[14772, 33649,  9324, 33743, 11666,  1619,  6599, 27799, 31215,  8591,
         24676, 28778,  6599, 27799,   355,   396, 29634,  1619,  6599, 27799,
          8358,   277,  6765,   979, 10205,   401,    78,  5259,   934, 15818,
           551,  1288,  3175, 4871